In [ ]:
import pymongo
import csv
import random
import string  
import bson

from dotenv.main import load_dotenv
import os
load_dotenv()
mongoUrl = os.environ['MONGO_URL']
dbName = os.environ['DB_NAME']

In [ ]:
def generate_object_id():
    return bson.ObjectId()

In [ ]:
def generate_password():
    # You can implement your password generation logic here
    # For simplicity, let's generate a random 8-character password
    return '$2b$10$MGhPmw7y/ZfcrY77cdF5guNg4Ohj6MY1vAWxxq2w8p70VpcWduwCe'

In [ ]:
def parse_coordinates(coordinates_str):
    parts = coordinates_str.strip('POINT()').split()
    return [float(parts[1]), float(parts[0])]

In [ ]:
client = pymongo.MongoClient(mongoUrl)
db = client[dbName]  # Replace with your database name
users_collection = db["users"]  # Collection for user data
locations_collection = db["locations"]

In [ ]:
countries = [
    "Nepal",
    "India",
    "China",
    "United States",
    "Canada",
    "Australia",
    "United Kingdom",
    "Germany",
    "France",
    "Japan",
    "Brazil",
]

In [ ]:
# Step 1: Generate User Data
for _ in range(100):
    user = {
        "_id": generate_object_id(),
        "username": f"user{_}",
        "password": generate_password(),
        "firstname": f"First{_}",
        "lastname": f"Last{_}",
        "follower": [],
        "following": [],
        "country": random.choice(countries),
        "livesIn": random.choice(countries),
        "profilePicture": "",
        "coverPicture": "",
        "locations": None  # We'll update this later
    }
    users_collection.insert_one(user)

print("Generated 100 user records.")

In [ ]:
# Step 2: Read and Transform Location Data from CSV (5000 places)
with open("./spots.csv", "r") as csvfile:
    csvreader = csv.DictReader(csvfile)
    for index, row in enumerate(csvreader):
        if index >= 500:  # Stop after reading 5000 places
            break

        location = {
            "_id": generate_object_id(),
            "name": row["spotname"],
            "coordinate": parse_coordinates(row["geo-coordinates"]),
            "ratings": []
        }
        locations_collection.insert_one(location)

print("Inserted 500 location data from CSV.")

In [ ]:
# Step 3: Link User and Location Data
users = list(users_collection.find({}))
for user in users:
    user_locations = random.sample(locations_collection.distinct("_id"), random.randint(10, 15))
    users_collection.update_one(
        {"_id": user["_id"]},
        {"$set": {"locations": user_locations}}
    )


In [ ]:
# Step 4: Add Ratings to Locations
users = list(users_collection.find({}))
for user in users:
    user_id = user["_id"]
    if user["locations"]:
        for location_id in user["locations"]:
            location = locations_collection.find_one({"_id": location_id})
            if location:
                existing_rating = next((r for r in location["ratings"] if r["user_id"] == user_id), None)
                if existing_rating:
                    # Update the existing rating if it exists
                    existing_rating["rate"] = random.randint(1, 5)
                    existing_rating["review"] = f"Updated review by {user['username']} for {location['name']}"
                    locations_collection.update_one(
                        {"_id": location["_id"], "ratings.user_id": user_id},
                        {"$set": {"ratings.$": existing_rating}}
                    )
                else:
                    # Create a new rating if it doesn't exist
                    new_rating = {
                        "user_id": user_id,
                        "rate": random.randint(1, 5),
                        "review": f"Review by {user['username']} for {location['name']}"
                    }
                    locations_collection.update_one(
                        {"_id": location["_id"]},
                        {"$push": {"ratings": new_rating}}
                    )

print("Added or updated ratings and reviews by users for their respective locations.")
